In [1]:
from pathlib import Path
import pandas as pd

Path.cwd()

PosixPath('/home/jovyan/2022_10_recall_mood/src')

In [2]:
CONVERSATIONS = Path.cwd().parent / 'data' / 'coded conversations'
RECALLS = Path.cwd().parent / 'data' / 'coded recalls'
CONDITIONS = Path.cwd().parent / 'data' / 'tracker.xlsx'

## Conversations

Dictionary of conversations texts. Each participant is a key, with the conversation they participated in as a value.
Conversation is duplicated, once for each participant who participated in it.

There are some unknown extension errors that need to be explored.

In [19]:
convos = {}
for f in CONVERSATIONS.iterdir():
    participant_id_1, participant_id_2 = f.stem.split('_')[1].split('&')
    tempdf = pd.read_excel(f, engine='openpyxl', header=None, usecols=[1], dtype=pd.StringDtype(), verbose=False).squeeze('columns')
    conversation_text = ' '.join(tempdf.astype(str))
    convos[participant_id_1] = conversation_text
    convos[participant_id_2] = conversation_text

/home/jovyan/conda_envs/spacy/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/home/jovyan/conda_envs/spacy/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/home/jovyan/conda_envs/spacy/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/home/jovyan/conda_envs/spacy/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/home/jovyan/conda_envs/spacy/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/home/jovy

## Recalls

Dictionary of recalls.

I pull text from the 'Coded' sheet of the file and use the 'Recall IU' column.

One unknown extension error needs to addressed.

In [20]:
recalls = {}
for f in RECALLS.iterdir():
    participant_id = f.stem[6:] # first 6 characters are 'Recall'. We want the id_num that follows.
    tempdf = pd.read_excel(f, engine='openpyxl', header=0, sheet_name='Coded', usecols=['Recall IU'], dtype=pd.StringDtype(), verbose=False).squeeze('columns')
    text = ' '.join(tempdf.astype(str))
    recalls[participant_id] = text

/home/jovyan/conda_envs/spacy/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [27]:
print(convos['1'][:100])
print(recalls['1'][:50])

I'm Spencer by the way I'm Jake Nice to meet you Alright so I think we can start with living in Nash
We can start with  living in Nashville I live  on 


## Conditions

In [23]:
dtypes = {'SubjectID': str,
          'ConditionExpt': pd.Categorical,
          'ConditionSub': pd.Categorical,
         }

df = pd.read_excel(CONDITIONS, engine='openpyxl', sheet_name='Conditions', dtype=dtypes, skiprows=[123], verbose=False)
df.head()

,Date,Experimenter,Subject Initials,SubjectID,ConditionExpt,ConditionSub,Credit,Notes
0,2018-11-30 00:00:00,Evgeniia,JM,1,Incongruent,Happy,1,NaN
1,2018-11-30 00:00:00,Evgeniia,SL,2,Incongruent,Sad,1,NaN
2,2018-12-13 00:00:00,Evgeniia,CB,3,Congruent,Happy,0,NaN
3,2018-12-13 00:00:00,Evgeniia,SG,4,Congruent,Happy,0,NaN
4,2019-01-16 00:00:00,Evgeniia,MB,5,Congruent,Sad,0,NaN


In [24]:
out = pd.merge(df, pd.Series(convos, name='convos'), left_on='SubjectID', right_index=True)
out = pd.merge(out, pd.Series(recalls, name='recalls'), left_on='SubjectID', right_index=True)

In [25]:
out.to_csv('../data/recall_mood_data.csv')

## To Discuss

Text cleaning steps
1. remove text between carots <>. These are annotator notes.
2. Remove asterisks and ellipses?

Also need to verify that the text I am pulling is faithful to the transcript. I did not spend much time verifying this.

Participants 117 and 118 are missing. 